#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [26]:
import pandas as pd
import pymongo

In [27]:
client = pymongo.MongoClient("mongo:27017")
database = client['exercices']
collection = database['kickstarter']

In [28]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head(15)

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225
5,1771789139,Room For Growth!,Couture,Fashion,USD,2016-05-02,2000,2016-04-11 18:15:00,6,failed,2,US,6,6.000000
6,1301627822,Build a mini Udemy with Laravel and Vuejs,Web,Technology,CAD,2018-01-11,3000,2017-12-12 01:37:26,155,live,7,CA,15.564,123.811806
7,881336601,Applitizer,Software,Technology,HKD,2017-08-20,100000,2017-07-21 16:50:05,1000,failed,1,HK,128.085,127.831467
8,1669374237,"Help I Am Wolves fund their first album ""ABCD""...",Music,Music,EUR,2017-12-04,1000,2017-10-27 16:55:47,1465,successful,46,BE,0,1720.210418
9,556821416,Eco Bamboo Underwear funding....GOT WOOD ?,Fashion,Fashion,GBP,2012-12-13,9500,2012-11-13 10:58:34,1212,failed,43,GB,1928.14,1953.672808


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [29]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [30]:
df_ks = df_ks.drop(df_ks[df_ks["backers"] == "successful"].index)

df_ks = df_ks.astype({"backers":"int64", "pledged":"float", "usd pledged":"float"})
df_ks["launched"] = pd.to_datetime(df_ks["launched"])
df_ks["deadline"] = pd.to_datetime(df_ks["deadline"])
df_ks["launched"] = df_ks["launched"].apply(lambda x : x.year)
df_ks["deadline"] = df_ks["deadline"].apply(lambda x : x.year)

### Importer les données

In [31]:
collection.delete_many({})
collection.insert_many(df_ks.to_dict("records"))

## Question 1  

In [32]:
cur = collection.find().sort([("pledged",-1)]).limit(5)
for document in cur :
    print('-----')
    print(document)

-----
{'_id': ObjectId('61f14dd0d137543ba3cf5796'), 'ID': 342886736, 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': 2014, 'goal': '50000.0', 'launched': 2014, 'pledged': 13285226.36, 'state': 'successful', 'backers': 62642, 'country': 'US', 'usd pledged': 13285226.36, 'usd_pledged_real': 13285226.36}
-----
{'_id': ObjectId('61f14dd0d137543ba3cfb28f'), 'ID': 2103598555, 'name': 'Pebble 2, Time 2 + All-New Pebble Core', 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': 2016, 'goal': '1000000.0', 'launched': 2016, 'pledged': 12779843.49, 'state': 'successful', 'backers': 66673, 'country': 'US', 'usd pledged': 12779843.49, 'usd_pledged_real': 12779843.49}
-----
{'_id': ObjectId('61f14dd0d137543ba3cf63cd'), 'ID': 2111201788, 'name': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA', 'category': 'Product Design', 'main_category': 'Design', 

## Question 2

In [33]:
collection.count_documents({"state":"successful"})

53040

## Question 3

In [34]:
cur = collection.aggregate([{"$group" : {"_id" :"$category" , "number" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Childrenswear', 'number': 192},
 {'_id': 'Stationery', 'number': 90},
 {'_id': 'Video Art', 'number': 65},
 {'_id': 'Sculpture', 'number': 737},
 {'_id': 'Accessories', 'number': 1204},
 {'_id': 'Theater', 'number': 2786},
 {'_id': 'Video Games', 'number': 4799},
 {'_id': 'Family', 'number': 130},
 {'_id': 'Chiptune', 'number': 10},
 {'_id': 'DIY', 'number': 480},
 {'_id': 'Journalism', 'number': 672},
 {'_id': "Children's Books", 'number': 2686},
 {'_id': 'Metal', 'number': 274},
 {'_id': 'Camera Equipment', 'number': 165},
 {'_id': 'Rock', 'number': 2707},
 {'_id': 'Action', 'number': 282},
 {'_id': 'Mobile Games', 'number': 650},
 {'_id': 'Puzzles', 'number': 95},
 {'_id': 'Zines', 'number': 144},
 {'_id': 'Flight', 'number': 158},
 {'_id': 'Interactive Design', 'number': 146},
 {'_id': 'Typography', 'number': 44},
 {'_id': 'Letterpress', 'number': 24},
 {'_id': 'Art', 'number': 3358},
 {'_id': 'Movie Theaters', 'number': 90},
 {'_id': 'Publishing', 'number': 2332},
 {'_id

## Question 4

In [35]:
collection.count_documents({"$and":[{"launched":{"$lt":2016}} , {"country" : "FR"}]})

330

## Question 5

In [37]:
cur = collection.find({"$and":[{"goal":{"$gt":200000}} , {"country" :"US"}]})
for document in cur :
    print('-----')
    print(document)

-----
{'_id': ObjectId('61f14dcfd137543ba3ce2d14'), 'ID': 866634482, 'name': 'A CALL TO ADVENTURE', 'category': 'Film & Video', 'main_category': 'Film & Video', 'currency': 'USD', 'deadline': 2012, 'goal': 287000.0, 'launched': 2012, 'pledged': 1465.0, 'state': 'failed', 'backers': 11, 'country': 'US', 'usd pledged': 1465.0, 'usd_pledged_real': 1465.0}
-----
{'_id': ObjectId('61f14dcfd137543ba3ce2dc7'), 'ID': 993194166, 'name': 'Storybricks, the storytelling online RPG', 'category': 'Video Games', 'main_category': 'Games', 'currency': 'USD', 'deadline': 2012, 'goal': 250000.0, 'launched': 2012, 'pledged': 23680.54, 'state': 'failed', 'backers': 409, 'country': 'US', 'usd pledged': 23680.54, 'usd_pledged_real': 23680.54}
-----
{'_id': ObjectId('61f14dcfd137543ba3ce2dcf'), 'ID': 1147175344, 'name': 'Shine On New World', 'category': 'Theater', 'main_category': 'Theater', 'currency': 'USD', 'deadline': 2013, 'goal': 300000.0, 'launched': 2013, 'pledged': 12314.0, 'state': 'failed', 'backer

-----
{'_id': ObjectId('61f14dd0d137543ba3ce77be'), 'ID': 1427832145, 'name': 'SKYDANCE', 'category': 'Drinks', 'main_category': 'Food', 'currency': 'USD', 'deadline': 2014, 'goal': 500000.0, 'launched': 2014, 'pledged': 2951.0, 'state': 'failed', 'backers': 6, 'country': 'US', 'usd pledged': 2951.0, 'usd_pledged_real': 2951.0}
-----
{'_id': ObjectId('61f14dd0d137543ba3ce781b'), 'ID': 1742121154, 'name': 'Skyteboard 3G Quadcopter with Fatdoor Social Network Connect', 'category': 'Flight', 'main_category': 'Technology', 'currency': 'USD', 'deadline': 2014, 'goal': 300000.0, 'launched': 2014, 'pledged': 3465.0, 'state': 'failed', 'backers': 17, 'country': 'US', 'usd pledged': 3465.0, 'usd_pledged_real': 3465.0}
-----
{'_id': ObjectId('61f14dd0d137543ba3ce7862'), 'ID': 763427662, 'name': 'Velvet Blu Mediterranean Restaurant', 'category': 'Food', 'main_category': 'Food', 'currency': 'USD', 'deadline': 2015, 'goal': 350000.0, 'launched': 2015, 'pledged': 0.0, 'state': 'failed', 'backers': 0

{'_id': ObjectId('61f14dd1d137543ba3d07422'), 'ID': 205248352, 'name': 'Unique B&B by preserving historic MCM Jap. Prairie ranch', 'category': 'Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': 2015, 'goal': 450000.0, 'launched': 2015, 'pledged': 0.0, 'state': 'failed', 'backers': 0, 'country': 'US', 'usd pledged': 0.0, 'usd_pledged_real': 0.0}
-----
{'_id': ObjectId('61f14dd1d137543ba3d0745e'), 'ID': 1912919329, 'name': 'Cancer Cure-The Movie-Big Pharma, Russian Mafia, One Doctor.', 'category': 'Film & Video', 'main_category': 'Film & Video', 'currency': 'USD', 'deadline': 2014, 'goal': 500000.0, 'launched': 2014, 'pledged': 1.0, 'state': 'canceled', 'backers': 1, 'country': 'US', 'usd pledged': 1.0, 'usd_pledged_real': 1.0}
-----
{'_id': ObjectId('61f14dd1d137543ba3d0747b'), 'ID': 203543199, 'name': 'The Essence Within Game Project', 'category': 'Video Games', 'main_category': 'Games', 'currency': 'USD', 'deadline': 2013, 'goal': 250000.0, 'launched': 2013, 'pledged'

## Question 6 

In [41]:
collection.create_index([("name", "text")])
cur = collection.find({"$text": { "$search": "Sport" }})
for document in cur :
    print('-----')
    print(document)

-----
{'_id': ObjectId('61f14dd0d137543ba3cedbc7'), 'ID': 1875366029, 'name': 'Sport Smart. A New Genre of Sports TV. Sport Fans Unite!!!', 'category': 'Webseries', 'main_category': 'Film & Video', 'currency': 'USD', 'deadline': 2011, 'goal': 4500.0, 'launched': 2010, 'pledged': 25.0, 'state': 'failed', 'backers': 1, 'country': 'US', 'usd pledged': 25.0, 'usd_pledged_real': 25.0}
-----
{'_id': ObjectId('61f14dcfd137543ba3ce5da9'), 'ID': 1126822169, 'name': 'Frey Sports App - We connect sports people.', 'category': 'Apps', 'main_category': 'Technology', 'currency': 'DKK', 'deadline': 2017, 'goal': 25000.0, 'launched': 2016, 'pledged': 0.0, 'state': 'failed', 'backers': 0, 'country': 'DK', 'usd pledged': 0.0, 'usd_pledged_real': 0.0}
-----
{'_id': ObjectId('61f14dd1d137543ba3d04427'), 'ID': 1081541783, 'name': 'Daily Fantasy Sports | Sports Analytics Platform | DFS', 'category': 'Web', 'main_category': 'Technology', 'currency': 'USD', 'deadline': 2017, 'goal': 33750.0, 'launched': 2017, 